In [1]:
import numpy as np
import pandas as pd
import pickle
import time

In [2]:
myfile = open('pickles/DI.pkl', 'rb')
X = pickle.load(myfile)
myfile.close()
print X.head()

myfile = open('pickles/used_column_headers.pkl', 'rb')
column_headers = pickle.load(myfile)
myfile.close()
print column_headers

myfile = open('pickles/ydf.pkl', 'rb')
y = pickle.load(myfile)
myfile.close()
print y.head()

0    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
2    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
3    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
4    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
Name: data, dtype: object
Index([u'time', u'2R', u'2G', u'2B', u'3R', u'3G', u'3B', u'4R', u'4G', u'4B',
       ...
       u'76B', u'77R', u'77G', u'77B', u'78R', u'78G', u'78B', u'79R', u'79G',
       u'79B'],
      dtype='object', length=220)
           classification gram  detection
0        Escherichia_coli    n          1
1        Escherichia_coli    n          1
2  Pseudomonas_aeruginosa    n          1
3  Pseudomonas_aeruginosa    n          1
4  Pseudomonas_aeruginosa    n          1


In [3]:
Ncolors = 3
Nsamp = len(X)
Ntimes = 60

col_nums = list(set([int(x[:-1]) for x in column_headers[1:]]))
Nspots = len(col_nums)

tidxs = y.index.values

# Convert series to frame
Z = X.to_frame()

# flatten arrays in data frame
start = time.time()
print 'Flattening'
final = np.array(Z['data'].iloc[0])
for i in xrange(1, Nsamp):
    final = np.vstack((final, Z['data'].iloc[i]))
    
end = time.time()
print 'Flattened %d trials in %d secs' % (Nsamp, (end-start))

Flattening
Flattened 1078 trials in 74 secs


In [4]:
print final.shape, Nsamp*Ntimes, 1+Ncolors*Nspots

(114771, 220) 64680 220


In [5]:
data = final[:,1:]

In [6]:
data = data.reshape((Ntimes*Nsamp*Nspots,3))
print data.shape, Ntimes*Nsamp, Ntimes*Nsamp*Nspots

ValueError: total size of new array must be unchanged

In [ ]:
df = pd.DataFrame(data=data, columns=['R', 'G', 'B'])
df.head()

In [ ]:
df['spot'] = np.array([x for x in col_nums]*(Nsamp*Ntimes)).flatten()
df.head()

In [ ]:
df['trial'] = np.array([[x]*(Ntimes*Nspots) for x in tidxs]).flatten()
df['detection'] = np.array([[y.ix[tidxs[x]]['detection']]*(Ntimes*Nspots) for x in range(Nsamp)]).flatten()
df['gram'] = np.array([[y.ix[tidxs[x]]['gram']]*(Ntimes*Nspots) for x in range(Nsamp)]).flatten()
df['classification'] = np.array([[y.ix[tidxs[x]]['classification']]*(Ntimes*Nspots) for x in range(Nsamp)]).flatten()
df.head()

In [ ]:
df['time'] = np.array([[[20*x]*Nspots for x in range(Ntimes)]*Nsamp]).flatten()
df.head()
df.iloc[72:75]

In [ ]:
# Experiment with time re-scaling
newdf = pd.DataFrame(final)

In [ ]:
newdf.columns = column_headers
newdf.head()

In [ ]:
print len(newdf), Nsamp*Ntimes

In [ ]:
# trial index
newdf['trial'] = np.array([[x]*(Ntimes) for x in tidxs]).flatten()
newdf['detection'] = np.array([[y.ix[tidxs[x]]['detection']]*(Ntimes) for x in range(Nsamp)]).flatten()
newdf['gram'] = np.array([[y.ix[tidxs[x]]['gram']]*(Ntimes) for x in range(Nsamp)]).flatten()
newdf['classification'] = np.array([[y.ix[tidxs[x]]['classification']]*(Ntimes) for x in range(Nsamp)]).flatten()
newdf.head()

In [ ]:
Z = X.to_frame()
Z['trial'] = y.index.values
Z.head()

In [ ]:
# randomly assign a trigger time
# assume it varies around 18 - 6 hours - with variance 9 - 3 hours
triggers = 18 + np.random.randn(len(Z))*5
# this would be calculated...
Z['trigger'] = triggers
Z.head()

## Assuming trigger is known, how to I rearrange this? ##

In [145]:
# rescale the t axis
# Z['data2'] = Z.apply(lambda x: (x['trigger'] - x['data'][:,0])/20*20, axis=1)
a = Z.iloc[0:3]

a.head()

,data,trial,trigger
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0,19.688072
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1,22.185134
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2,15.028030


In [141]:
def hstacker(x, trigger):
    return np.hstack(((x[:,0]-trigger*np.ones(x.shape[1])).reshape(-1,1), x[:,1:]))
print a.head()

a['data3'] = hstacker(x['data'], x['trigger'])
# a['data2'] = (-a['trigger'] + a['data'][:,0])
# a['data3'] = a.apply(lambda x: np.hstack((a['data3'].reshape(-1,1), x['data'][:,1:])))

                                                data  trial    trigger
0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...      0  19.688072
1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...      1  22.185134
2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...      2  15.028030


TypeError: 'int' object has no attribute '__getitem__'

In [155]:
b = [np.hstack(((x[:,0] - trigger*np.ones(Ntimes)).reshape(-1,1),x[:,1:])) for x, trigger in zip(a['data'].values, a['trigger'].values)]
print len(b)

3


In [128]:
b = np.hstack(((b[:,0]-150).reshape(-1,1), b[:,1:]))
b

array([[ -150.,     0.,     0., ...,     0.,     0.,     0.],
       [ -130.,     0.,     0., ...,     0.,     0.,     0.],
       [ -110.,     0.,     0., ...,     0.,     0.,     0.],
       ..., 
       [  990.,   -28.,  -112., ...,   -20.,   -18.,   -24.],
       [ 1010.,   -35.,  -119., ...,   -20.,   -18.,   -25.],
       [ 1030.,   -31.,  -116., ...,   -22.,   -14.,   -29.]])

In [125]:
d = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
triggers[0]

19.688071950091054

In [122]:
print d
np.hstack((d[:,1:], d[:,0].reshape(-1,1)))

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]


array([[ 2,  3,  1],
       [ 5,  6,  4],
       [ 8,  9,  7],
       [11, 12, 10]])

In [119]:
d[:,1:]

array([[ 2,  3],
       [ 5,  6],
       [ 8,  9],
       [11, 12]])

In [121]:
d[:,0].reshape(-1,1)

array([[ 1],
       [ 4],
       [ 7],
       [10]])